In [1]:
def run_simulation_2D(x, y, nx, ny, Lx, Ly, cx, cy, sx, sy):
    dx, dy = Lx/(nx-1), Ly/(ny-1)
    dt = 1
    tend = 1200
    t = 0

    cfl_x, cfl_y = cx * dt/dx, cy * dt/dy
    diff_x, diff_y = sx * dt/dx**2, sy * dt/dy**2

    u = np.zeros((nx+2, ny+2))
    sol = []
    source_x, source_y = nx // 2, ny // 2
    Q = 1e-6
    
    while t < tend:
        unew = u.copy()
        sol.append(u[1:-1, 1:-1])

         # Advection (Upwind Scheme)
        unew[1:-1, 1:-1] -= cfl_x * (u[1:-1, 1:-1] - u[1:-1, :-2])
        unew[1:-1, 1:-1] -= cfl_y * (u[1:-1, 1:-1] - u[:-2, 1:-1])
    
        # Diffusion (Central Differencing)
        unew[1:-1, 1:-1] += diff_x * (u[1:-1, 2:] - 2*u[1:-1, 1:-1] + u[1:-1, :-2])
        unew[1:-1, 1:-1] += diff_y * (u[2:, 1:-1] - 2*u[1:-1, 1:-1] + u[:-2, 1:-1])

        # Source Term
        unew[source_x, source_y] += Q * dt

        # Additional Source Points (forming a small area)
        offsets = [(-1, -1), (-1, 1), (1, -1), (1, 1), (-1, 0), (1, 0), (0, -1), (0, 1)]
        for dx, dy in offsets:
            unew[source_x + dx, source_y + dy] += Q * dt

        u = unew
        t += dt

    '''
    We transpose the axis the solution. Such that:
    - Axis 0: x
    - Axis 1: y
    - Axis 2: time
    Interpretation: For each x-grid, we have the concentration of each y-grid over time.
    Essentially, the (ny, 1200) array represents the concentration at each y over the time.
    So we have an array of size 1200 for each y. (A curve)
    '''
    sol = np.transpose(sol, (1, 2, 0))
    return np.array(sol)

In [2]:
from joblib import Parallel, delayed
import numpy as np
import time as time

nx, ny= 51, 51  # Grid points
Lx, Ly = 5000, 5000  # Domain size in meters
x = np.linspace(-2500, 2500, nx)  # Centered at (0,0)
y = np.linspace(-2500, 2500, ny)
n = 50
cx, cy = np.random.RandomState().uniform(0, 10, n), np.random.RandomState().uniform(0, 10, n)
sx, sy = np.random.RandomState().uniform(0, 1, n), np.random.RandomState().uniform(0, 1, n)
num_cores = -1

start_time = time.time()
results = Parallel(n_jobs=num_cores)(
    delayed(run_simulation_2D)(x, y, nx, ny, Lx, Ly, cx[i], cy[i], sx[i], sy[i])
    for i in range(n)
)
end_time = time.time()

print(f"Simulation took: {end_time-start_time}")

Simulation took: 2.3374218940734863


## Applying Distance Metrics

We try to implement the same way as we have for the 1D problem, and adjust if there are any issues.

Because the shape of the results would be in 4D (n, Nx, Ny, time), it would be infeasible to try and solve everything all at once.

However, parallelisation can still be utilised.

The distances should be modified such that it computes the distance **for each spatial location** across time first (outputting a 51x51 matrix), with each (i, j) representating the distance at that point, and then output an average (?).

The original distance metrics were desgined so that it computes the distance between each column.
- This is because each column represented one solution.

In [3]:
observed = np.load("test.npy")
observed.shape

(51, 51, 1200)

### Wasserstein

In [9]:
### Original ###
def wasserstein_distance(simulated_sample: np.ndarray, observed_sample: np.ndarray) -> float:
    # Mean Difference between simulated and observed
    simulated_sorted = np.sort(simulated_sample, axis=0)
    observed_sorted = np.sort(observed_sample, axis=0)
    distance = np.mean(np.abs(simulated_sorted - observed_sorted), axis=0)

    return distance

### Modified ###
def wasserstein_distance_3D(simulated_sample: np.ndarray, observed_sample: np.ndarray) -> np.ndarray:
    """
    Compute the Wasserstein distance between two (51, 51, 1200) shaped arrays 
    along the time dimension.

    Sorted along the time axis, we now have a (51, 51, 1200) array, with each 1200-sized array sorted ascendingly 

    Returns a (51, 51) array of distances for each spatial location.
    """
    # Sort along the time axis (axis=2)
    simulated_sorted = np.sort(simulated_sample, axis=2)
    observed_sorted = np.sort(observed_sample, axis=2)

    # Compute the mean absolute difference along the time axis
    distance = np.mean(np.abs(simulated_sorted - observed_sorted), axis=2)

    return distance 

In [17]:
start = time.time()
wass = Parallel(n_jobs=num_cores)(
    delayed(wasserstein_distance_3D)(results[i], observed)
    for i in range(n)
)
end = time.time()
end-start

1.4817471504211426

### CvMD

In [28]:
### Original ###
def cramer_von_mises(simulated_sample: np.ndarray, observed_sample: np.ndarray) -> float:
    if len(simulated_sample) != len(observed_sample):
        return "Size of samples not equal."
    
    nrow = simulated_sample.shape[0]
    ncol = simulated_sample.shape[1]
    combined = np.concatenate((simulated_sample, observed_sample))
    # Find corresponding ranks in h associated with simulated/observed
    combined_rank = np.argsort(combined, axis=0)+1
    simulated_rank = combined_rank[:nrow]
    observed_rank = combined_rank[nrow:]

    # Calculate distance
    idx = np.tile(np.arange(1, nrow+1), (ncol, 1)).T
    observed_sum = np.sum((observed_rank - idx)**2, axis=0)
    simulated_sum = np.sum((simulated_rank - idx)**2, axis=0)
    rank_sum = nrow * (observed_sum + simulated_sum)
    distance = rank_sum / (2*nrow**3) - (4*nrow**2 - 1)/(12*nrow)

    return distance

### Modified ###
def cramer_von_mises_3d(simulated_sample: np.ndarray, observed_sample: np.ndarray) -> np.ndarray:
    '''
    Outputs a (51, 51), representing CvMD at each grid point.
    '''
    if simulated_sample.shape != observed_sample.shape:
        raise ValueError("Shape of samples not equal.")
    
    x_dim, y_dim, n_samples = simulated_sample.shape
    cvm_matrix = np.zeros((x_dim, y_dim))

    # Comparing the (1200,)-shaped array for each (x, y) grid point
    for i in range(x_dim):
        for j in range(y_dim):
            sim = simulated_sample[i, j, :]
            obs = observed_sample[i, j, :]

            # Combining the two arrays. (2400,)-shaped array
            combined = np.concatenate((sim, obs))
            # np.argsort(combined) gives us indicies that would sort the combined array in ascending order.
            # applying np.argosrt again gives us the rank of each element in the original array.
            combined_rank = np.argsort(np.argsort(combined)) + 1

            # First half of the combined rank is simulated by definition of combined.
            sim_rank = combined_rank[:n_samples]
            obs_rank = combined_rank[n_samples:]

            # Calculation for CvMD
            idx = np.arange(1, n_samples + 1)
            obs_sum = np.sum((obs_rank - idx) ** 2)
            sim_sum = np.sum((sim_rank - idx) ** 2)
            
            rank_sum = n_samples * (obs_sum + sim_sum)
            distance = rank_sum / (2 * n_samples**3) - (4 * n_samples**2 - 1) / (12 * n_samples)
            
            cvm_matrix[i, j] = distance
    
    return cvm_matrix

In [29]:
start = time.time()
cvmd = Parallel(n_jobs=num_cores)(
    delayed(cramer_von_mises_3d)(results[i], observed)
    for i in range(n)
)
end = time.time()
end-start

2.9667017459869385

### Functional Frechet Distance

In [4]:
def _c(ca, i, j, p, q):

    if ca[i, j] > -1:
        return ca[i, j]
    elif i == 0 and j == 0:
        ca[i, j] = np.linalg.norm(p[i]-q[j])
    elif i > 0 and j == 0:
        ca[i, j] = max(_c(ca, i-1, 0, p, q), np.linalg.norm(p[i]-q[j]))
    elif i == 0 and j > 0:
        ca[i, j] = max(_c(ca, 0, j-1, p, q), np.linalg.norm(p[i]-q[j]))
    elif i > 0 and j > 0:
        ca[i, j] = max(
            min(
                _c(ca, i-1, j, p, q),
                _c(ca, i-1, j-1, p, q),
                _c(ca, i, j-1, p, q)
            ),
            np.linalg.norm(p[i]-q[j])
            )
    else:
        ca[i, j] = float('inf')

    return ca[i, j]


def frdist(p, q):
    """
    Computes the discrete Fréchet distance between
    two curves. The Fréchet distance between two curves in a
    metric space is a measure of the similarity between the curves.
    The discrete Fréchet distance may be used for approximately computing
    the Fréchet distance between two arbitrary curves,
    as an alternative to using the exact Fréchet distance between a polygonal
    approximation of the curves or an approximation of this value.

    This is a Python 3.* implementation of the algorithm produced
    in Eiter, T. and Mannila, H., 1994. Computing discrete Fréchet distance.
    Tech. Report CD-TR 94/64, Information Systems Department, Technical
    University of Vienna.
    http://www.kr.tuwien.ac.at/staff/eiter/et-archive/cdtr9464.pdf

    Function dF(P, Q): real;
        input: polygonal curves P = (u1, . . . , up) and Q = (v1, . . . , vq).
        return: δdF (P, Q)
        ca : array [1..p, 1..q] of real;
        function c(i, j): real;
            begin
                if ca(i, j) > −1 then return ca(i, j)
                elsif i = 1 and j = 1 then ca(i, j) := d(u1, v1)
                elsif i > 1 and j = 1 then ca(i, j) := max{ c(i − 1, 1), d(ui, v1) }
                elsif i = 1 and j > 1 then ca(i, j) := max{ c(1, j − 1), d(u1, vj) }
                elsif i > 1 and j > 1 then ca(i, j) :=
                max{ min(c(i − 1, j), c(i − 1, j − 1), c(i, j − 1)), d(ui, vj ) }
                else ca(i, j) = ∞
                return ca(i, j);
            end; /* function c */

        begin
            for i = 1 to p do for j = 1 to q do ca(i, j) := −1.0;
            return c(p, q);
        end.

    Parameters
    ----------
    P : Input curve - two dimensional array of points
    Q : Input curve - two dimensional array of points

    Returns
    -------
    dist: float64
        The discrete Fréchet distance between curves `P` and `Q`.

    Examples
    --------
    >>> from frechetdist import frdist
    >>> P=[[1,1], [2,1], [2,2]]
    >>> Q=[[2,2], [0,1], [2,4]]
    >>> frdist(P,Q)
    >>> 2.0
    >>> P=[[1,1], [2,1], [2,2]]
    >>> Q=[[1,1], [2,1], [2,2]]
    >>> frdist(P,Q)
    >>> 0
    """
    p = np.array(p, np.float64)
    q = np.array(q, np.float64)

    len_p = len(p)
    len_q = len(q)

    if len_p == 0 or len_q == 0:
        raise ValueError('Input curves are empty.')
        
    ca = (np.ones((len_p, len_q), dtype=np.float64) * -1)

    dist = _c(ca, len_p-1, len_q-1, p, q)
    return dist

In [8]:
start = time.time()
frdist(results[0][0, 0], observed[0, 0])
end = time.time()
print(end-start)

7.418493032455444


One iteration of this takes ~7.5 seconds, and we have to go through 51x51 of this just for one simulation. 

In [9]:
from joblib import Parallel, delayed

frechet_results = np.zeros((51, 51))
start = time.time()
for i in range(51):
    for j in range(51):
        frechet_results[i, j] = frdist(results[0][i, j], observed[i, j])
end = time.time()
print(end-start)

# Aggregate results
max_distance = np.max(frechet_results)
mean_distance = np.mean(frechet_results)

print(f"Max Fréchet Distance: {max_distance}")
print(f"Mean Fréchet Distance: {mean_distance}")

KeyboardInterrupt: 

This is the general workflow, but it will take too long to compute because of the double loops.